In [1]:
import pandas as pd
import numpy as np

In [2]:
test_data = pd.read_csv('./test_data.csv')
test_data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [7]:
test_data.text[3]

'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'

In [52]:
nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivandemyanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
def delete_stopwords(text):
    """
    :param str text: Input text 
    :return List[str]: List of words
    """
    text = text.lower()
    text = re.sub(r'[^А-Яа-я ]', '', text).split(' ')
    text = [w for w in text if w not in set(STOPWORDS) and w != '']
    return " ".join(text)

In [71]:
delete_stopwords(test_data.text[3])

'зовут ангелина компания диджитал бизнес звоним поводу продления лицензии серым скоро срок заканчивается'

In [67]:
' '.join([ 'List','of',' string' ])

'List of  string'

In [72]:
df.text.apply(delete_stopwords)

0                                                   алло
1                                      алло здравствуйте
2                                            добрый день
3      зовут ангелина компания диджитал бизнес звоним...
4                                                    ага
                             ...                        
475       поводу виджетов ними обсудите конкретно продам
476    высылаю счет вами связи будут вопросы можете п...
477                                      спасибо спасибо
478                       созвонимся ага спасибо давайте
479                             свидания хорошего вечера
Name: text, Length: 480, dtype: object

In [73]:
df['text2'] = df.text.apply(delete_stopwords)

In [74]:
df

,dlg_id,line_n,role,text,NLTK,STANZA,NATASHA,PULLENTI,text2
0,0,0,client,Алло,0,entity: Алло\ttype: PER,0,NaN,алло
1,0,1,manager,Алло здравствуйте,0,entity: Алло\ttype: PER,"Name(first=None, last='здравствуйте', middle=N...",NaN,алло здравствуйте
2,0,2,client,Добрый день,0,0,"Name(first=None, last='Добрый', middle=None)Na...",NaN,добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина ( ангелина)score:1.0 \,0,"Name(first='ангелина', last=None, middle=None)...",NaN,зовут ангелина компания диджитал бизнес звоним...
4,0,4,client,Ага,0,entity: Ага\ttype: PER,0,NaN,ага
...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,0,0,"Name(first=None, last='По', middle=None)Name(f...",NaN,поводу виджетов ними обсудите конкретно продам
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,0,0,"Name(first=None, last='и', middle=None)Name(fi...",NaN,высылаю счет вами связи будут вопросы можете п...
477,5,140,client,Спасибо спасибо,0,0,"Name(first=None, last='Спасибо', middle=None)N...",NaN,спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,0,0,"Name(first=None, last='спасибо', middle=None)N...",NaN,созвонимся ага спасибо давайте


### Извлечение реплик, где менеджер представил себя, и имени менеджера

In [8]:
import nltk
import pymorphy2
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import stanza
from natasha import NamesExtractor
from pullenti_wrapper.processor import Processor, PERSON
from sklearn.metrics import f1_score
stanza.download('ru')
nltk.download('punkt')
stanza_nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
morph = pymorphy2.MorphAnalyzer()

2022-09-03 17:10:34 INFO: Downloading default packages for language: ru (Russian)...
2022-09-03 17:10:36 INFO: File exists: /Users/ivandemyanov/stanza_resources/ru/default.zip
2022-09-03 17:10:47 INFO: Finished downloading models and saved to /Users/ivandemyanov/stanza_resources.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ivandemyanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2022-09-03 17:10:49 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2022-09-03 17:10:49 INFO: Use device: cpu
2022-09-03 17:10:49 INFO: Loading: tokenize
2022-09-03 17:10:49 INFO: Loading: ner
2022-09-03 17:10:50 INFO: Done loading processors!


In [75]:
def name_recognize_stanza(text):
    result = ''
    global stanza_nlp
    doc = stanza_nlp(text)
    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == 'PER':
                result +=  f'entity: {ent.text}\ttype: {ent.type}' + " "
    if result != '':
        return result
    else: return 0

In [76]:
def name_recognize_natasha(text):
    global morph
    extractor = NamesExtractor(morph)
    matches = extractor(text)
    if matches != None:
        result = ''
        for index, match in enumerate(matches):
            if match != None:
                result += str(match.fact)
    if result !='':
        return result
    else:
        return 0

In [77]:
def name_recognize_pullenti(text):
    result = ''
    processor = Processor([PERSON])
    ner_result = processor(text)
    if ner_result.matches != []:
        for match in ner_result.matches:
            result += str(match)+ ' '
    if result != '':
        return result
    else: return 0

In [78]:
def name_recognize_nltk(text):
    prob_thresh = 0.4
    text = str(text)
    result = ''
    global morph
    for word in nltk.word_tokenize(text):
        
        if re.search("максим", word.lower()):
            result += 'Максим      (      максим)score:1.0 \\ '
        
        else:
            for p in morph.parse(word):
                if 'Name' in p.tag:
                    print(p, '\n', p.score)
                if ('Name' in p.tag and p.score >= prob_thresh) or ('Surn' in p.tag and p.score >= prob_thresh):
                    result += '{:<12}({:>12})score:{:0.3}'.format(word, p.normal_form, p.score)
                    result += ' \ '
    if result != '':
        return result
    else:
        return 0

In [128]:
def name_recognize_nltk(text):
    prob_thresh = 0.5
    text = str(text)
    result = ''
    global morph

    for word in nltk.word_tokenize(text):        
        for p in morph.parse(word):
            if ('Name' in p.tag and p.score >= prob_thresh) or ('Surn' in p.tag and p.score >= prob_thresh):
                result += '{:<12}({:>12})score:{:0.3}'.format(word, p.normal_form, p.score)
                result += ' \ '
    
    if result != '':
        return result
    else:
        return 0

In [ ]:
def dlg_lst(data):
    

In [ ]:
def pars

In [129]:
df['NLTK2'] = df.text2.apply(name_recognize_nltk)#[251]

In [324]:
for idx in np.unique(df.dlg_id):
    df[df.dlg_id == idx]

array([0, 1, 2, 3, 4, 5])

In [345]:
df[df.dlg_id == 0][df[df.dlg_id == 0].role == 'manager'][
    df[df.dlg_id == 0][df[df.dlg_id == 0].role == 'manager'].text2.apply(name_recognize_nltk) != 0]

,dlg_id,line_n,role,text,NLTK,STANZA,NATASHA,PULLENTI,text2,STANZA2,NATASHA2,NLTK2
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина ( ангелина)score:1.0 \,0,"Name(first='ангелина', last=None, middle=None)...",NaN,зовут ангелина компания диджитал бизнес звоним...,0,"Name(first='ангелина', last=None, middle=None)...",ангелина ( ангелина)score:1.0 \


In [348]:
df[df.dlg_id == 3][df[df.dlg_id == 3].role == 'manager'][
    df[df.dlg_id == 3][df[df.dlg_id == 3].role == 'manager'].text2.apply(name_recognize_nltk) != 0].NLTK

250             дмитрий     (     дмитрий)score:0.986 \ 
251               Максим      (      максим)score:1.0 \ 
253    дмитрий     (     дмитрий)score:0.986 \ анаста...
Name: NLTK, dtype: object

In [351]:
df[df.dlg_id == 3][df[df.dlg_id == 3].role == 'manager'][
    df[df.dlg_id == 3][df[df.dlg_id == 3].role == 'manager'].text2.apply(name_recognize_nltk) != 0].NLTK

250             дмитрий     (     дмитрий)score:0.986 \ 
251               Максим      (      максим)score:1.0 \ 
253    дмитрий     (     дмитрий)score:0.986 \ анаста...
Name: NLTK, dtype: object

In [133]:
df 

,dlg_id,line_n,role,text,NLTK,STANZA,NATASHA,PULLENTI,text2,STANZA2,NATASHA2,NLTK2
0,0,0,client,Алло,0,entity: Алло\ttype: PER,0,NaN,алло,0,0,0
1,0,1,manager,Алло здравствуйте,0,entity: Алло\ttype: PER,"Name(first=None, last='здравствуйте', middle=N...",NaN,алло здравствуйте,0,"Name(first=None, last='здравствуйте', middle=N...",0
2,0,2,client,Добрый день,0,0,"Name(first=None, last='Добрый', middle=None)Na...",NaN,добрый день,0,"Name(first=None, last='Добрый', middle=None)Na...",0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина ( ангелина)score:1.0 \,0,"Name(first='ангелина', last=None, middle=None)...",NaN,зовут ангелина компания диджитал бизнес звоним...,0,"Name(first='ангелина', last=None, middle=None)...",ангелина ( ангелина)score:1.0 \
4,0,4,client,Ага,0,entity: Ага\ttype: PER,0,NaN,ага,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,0,0,"Name(first=None, last='По', middle=None)Name(f...",NaN,поводу виджетов ними обсудите конкретно продам,0,"Name(first=None, last='По', middle=None)Name(f...",0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,0,0,"Name(first=None, last='и', middle=None)Name(fi...",NaN,высылаю счет вами связи будут вопросы можете п...,0,"Name(first=None, last='и', middle=None)Name(fi...",0
477,5,140,client,Спасибо спасибо,0,0,"Name(first=None, last='Спасибо', middle=None)N...",NaN,спасибо спасибо,0,"Name(first=None, last='Спасибо', middle=None)N...",0
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,0,0,"Name(first=None, last='спасибо', middle=None)N...",NaN,созвонимся ага спасибо давайте,0,"Name(first=None, last='спасибо', middle=None)N...",0


In [132]:
df['role' == ]

0                                                   алло
1                                      алло здравствуйте
2                                            добрый день
3      зовут ангелина компания диджитал бизнес звоним...
4                                                    ага
                             ...                        
475       поводу виджетов ними обсудите конкретно продам
476    высылаю счет вами связи будут вопросы можете п...
477                                      спасибо спасибо
478                       созвонимся ага спасибо давайте
479                             свидания хорошего вечера
Name: text2, Length: 480, dtype: object

In [138]:
df[df['role'] == 'manager']

,dlg_id,line_n,role,text,NLTK,STANZA,NATASHA,PULLENTI,text2,STANZA2,NATASHA2,NLTK2
1,0,1,manager,Алло здравствуйте,0,entity: Алло\ttype: PER,"Name(first=None, last='здравствуйте', middle=N...",NaN,алло здравствуйте,0,"Name(first=None, last='здравствуйте', middle=N...",0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина ( ангелина)score:1.0 \,0,"Name(first='ангелина', last=None, middle=None)...",NaN,зовут ангелина компания диджитал бизнес звоним...,0,"Name(first='ангелина', last=None, middle=None)...",ангелина ( ангелина)score:1.0 \
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,0,0,"Name(first=None, last='и', middle=None)Name(fi...",NaN,угу возможно рассмотрите другие варианты видит...,0,"Name(first=None, last='и', middle=None)Name(fi...",0
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,0,0,"Name(first=None, last='а', middle=None)Name(fi...",NaN,угу обращаете внимание выборе,0,"Name(first=None, last='а', middle=None)Name(fi...",0
11,0,11,manager,Что для вас приоритет,0,0,"Name(first=None, last='для', middle=None)",NaN,приоритет,0,"Name(first=None, last='для', middle=None)",0
...,...,...,...,...,...,...,...,...,...,...,...,...
472,5,135,manager,Так дмитрий,дмитрий ( дмитрий)score:0.986 \,0,"Name(first='дмитрий', last='Так', middle=None)",NaN,дмитрий,0,"Name(first='дмитрий', last='Так', middle=None)",дмитрий ( дмитрий)score:0.986 \
473,5,136,manager,Все записала тогда завтра ждите звонка,0,0,"Name(first=None, last='звонка', middle=None)",NaN,записала завтра ждите звонка,0,"Name(first=None, last='звонка', middle=None)",0
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,0,0,"Name(first=None, last='По', middle=None)Name(f...",NaN,поводу виджетов ними обсудите конкретно продам,0,"Name(first=None, last='По', middle=None)Name(f...",0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,0,0,"Name(first=None, last='и', middle=None)Name(fi...",NaN,высылаю счет вами связи будут вопросы можете п...,0,"Name(first=None, last='и', middle=None)Name(fi...",0


In [320]:
df.text[341]

'Вот мы просто нашли сами сейчас всю базу вот эти вот выкачали мы тогда предлагаем как если мы сейчас обнулим ее полностью да ну как бы очистим и значит вот смотрите вот тут дима там вопросы я не знаю или к вам или вы нас после'

In [321]:
df.role[341]

'client'

In [315]:
df.NLTK2[338]

'анастасия   (   анастасия)score:0.939 \\ '

In [292]:
df.line_n[253]

4

In [14]:
df = test_data.copy(deep=True)

In [16]:
test_data

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


In [99]:
df

,dlg_id,line_n,role,text,NLTK,STANZA,NATASHA,PULLENTI,text2,STANZA2,NATASHA2,NLTK2
0,0,0,client,Алло,0,entity: Алло\ttype: PER,0,NaN,алло,0,0,0
1,0,1,manager,Алло здравствуйте,0,entity: Алло\ttype: PER,"Name(first=None, last='здравствуйте', middle=N...",NaN,алло здравствуйте,0,"Name(first=None, last='здравствуйте', middle=N...",0
2,0,2,client,Добрый день,0,0,"Name(first=None, last='Добрый', middle=None)Na...",NaN,добрый день,0,"Name(first=None, last='Добрый', middle=None)Na...",0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина ( ангелина)score:1.0 \,0,"Name(first='ангелина', last=None, middle=None)...",NaN,зовут ангелина компания диджитал бизнес звоним...,0,"Name(first='ангелина', last=None, middle=None)...",ангелина ( ангелина)score:1.0 \
4,0,4,client,Ага,0,entity: Ага\ttype: PER,0,NaN,ага,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,0,0,"Name(first=None, last='По', middle=None)Name(f...",NaN,поводу виджетов ними обсудите конкретно продам,0,"Name(first=None, last='По', middle=None)Name(f...",0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,0,0,"Name(first=None, last='и', middle=None)Name(fi...",NaN,высылаю счет вами связи будут вопросы можете п...,0,"Name(first=None, last='и', middle=None)Name(fi...",0
477,5,140,client,Спасибо спасибо,0,0,"Name(first=None, last='Спасибо', middle=None)N...",NaN,спасибо спасибо,0,"Name(first=None, last='Спасибо', middle=None)N...",0
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,0,0,"Name(first=None, last='спасибо', middle=None)N...",NaN,созвонимся ага спасибо давайте,0,"Name(first=None, last='спасибо', middle=None)N...",0


In [121]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df[['NLTK2', 'text']])

                                                 NLTK2  \
0                                                    0   
1                                                    0   
2                                                    0   
3               ангелина    (    ангелина)score:1.0 \    
4                                                    0   
5                                                    0   
6                                                    0   
7                                                    0   
8                                                    0   
9                                                    0   
10                                                   0   
11                                                   0   
12                                                   0   
13                                                   0   
14                                                   0   
15                                                   0   
16            

In [102]:
df

,dlg_id,line_n,role,text,NLTK,STANZA,NATASHA,PULLENTI,text2,STANZA2,NATASHA2,NLTK2
0,0,0,client,Алло,0,entity: Алло\ttype: PER,0,NaN,алло,0,0,0
1,0,1,manager,Алло здравствуйте,0,entity: Алло\ttype: PER,"Name(first=None, last='здравствуйте', middle=N...",NaN,алло здравствуйте,0,"Name(first=None, last='здравствуйте', middle=N...",0
2,0,2,client,Добрый день,0,0,"Name(first=None, last='Добрый', middle=None)Na...",NaN,добрый день,0,"Name(first=None, last='Добрый', middle=None)Na...",0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина ( ангелина)score:1.0 \,0,"Name(first='ангелина', last=None, middle=None)...",NaN,зовут ангелина компания диджитал бизнес звоним...,0,"Name(first='ангелина', last=None, middle=None)...",ангелина ( ангелина)score:1.0 \
4,0,4,client,Ага,0,entity: Ага\ttype: PER,0,NaN,ага,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,0,0,"Name(first=None, last='По', middle=None)Name(f...",NaN,поводу виджетов ними обсудите конкретно продам,0,"Name(first=None, last='По', middle=None)Name(f...",0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,0,0,"Name(first=None, last='и', middle=None)Name(fi...",NaN,высылаю счет вами связи будут вопросы можете п...,0,"Name(first=None, last='и', middle=None)Name(fi...",0
477,5,140,client,Спасибо спасибо,0,0,"Name(first=None, last='Спасибо', middle=None)N...",NaN,спасибо спасибо,0,"Name(first=None, last='Спасибо', middle=None)N...",0
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,0,0,"Name(first=None, last='спасибо', middle=None)N...",NaN,созвонимся ага спасибо давайте,0,"Name(first=None, last='спасибо', middle=None)N...",0


### Company

In [7]:
from keybert import KeyBERT

In [6]:
!pip3 install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.7 MB/s 
     |████████████████████████████████| 235 kB 63.0 MB/s 
     |████████████████████████████████| 51 kB 9.2 MB/s 
     |████████████████████████████████| 4.7 MB 71.5 MB/s 
     |████████████████████████████████| 1.3 MB 58.7 MB/s 
     |████████████████████████████████| 120 kB 79.4 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=82797a82e11701c7debffae44a5449e95a804e71a3bfaea8cfa74f4a7e9ba088
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [8]:
kw_model = KeyBERT()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [19]:
keywords = kw_model.extract_keywords(test_data.text)

In [24]:
test_data.text[3]

'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'

In [47]:
kw_model.extract_keywords(test_data.text[3], keyphrase_ngram_range=(2, 3), stop_words=None,
                          use_mmr=True, diversity=0.5)

[('зовут ангелина компания', 0.767),
 ('поводу продления лицензии', 0.7222),
 ('скоро срок заканчивается', 0.6633),
 ('диджитал бизнес звоним', 0.6627),
 ('меня зовут', 0.6335)]

In [48]:
from sentence_transformers import SentenceTransformer

In [50]:
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [56]:
kw_model = KeyBERT(model=sentence_model)

In [62]:
kw_model.extract_keywords(test_data.text[3], keyphrase_ngram_range=(1, 3), stop_words=None,
                          use_mmr=True, diversity=0.2)

[('скоро срок заканчивается', 0.8642),
 ('диджитал бизнес звоним', 0.8543),
 ('компания диджитал бизнес', 0.8481),
 ('меня зовут ангелина', 0.8339),
 ('поводу продления лицензии', 0.8208)]

In [64]:
kw_model.extract_keywords(test_data.text[3], keyphrase_ngram_range=(1, 3), stop_words=None,
                          use_mmr=True, diversity=0.15)

[('скоро срок заканчивается', 0.8642),
 ('диджитал бизнес звоним', 0.8543),
 ('компания диджитал бизнес', 0.8481),
 ('меня зовут ангелина', 0.8339),
 ('поводу продления лицензии', 0.8208)]

In [61]:
kw_model.extract_keywords(test_data.text[3], keyphrase_ngram_range=(1, 3), stop_words=None,
                          use_maxsum=True)

[('по поводу продления', 0.7586),
 ('лицензии мы серым', 0.7701),
 ('бизнес звоним', 0.7716),
 ('вас скоро срок', 0.7918),
 ('ангелина компания диджитал', 0.8157)]

In [71]:
import stanza

stanza.download('russian') 
nlp = stanza.Pipeline('russian')
txt = test_data.text[3]
doc = nlp(txt)
for sent in doc.sentences: 
	for ent in sent.entities:
		print (ent.text,' ',ent.type)

INFO:stanza:"russian" is an alias for "ru"
INFO:stanza:Downloading default packages for language: ru (Russian)...
INFO:stanza:File exists: /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


INFO:stanza:"russian" is an alias for "ru"
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [1]:
test_data.text[3]

NameError: ignored

In [66]:
!pip3 install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 32.8 MB/s 
     |████████████████████████████████| 197 kB 78.6 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=ed782cfbaceacaebb69e20d8a8095f7c12b8660c467f5499666e423cf7acf36e
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [12]:
import spacy
import spacy.displacy as displacy

In [52]:
!pip install -U spacy
!python -m spacy download ru_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-02 14:11:10.131837: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

✘ No compatible package found for 'ru_core_web_sm' (spaCy v3.4.1)



In [36]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp = spacy.load("ru_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Меня PRON obj
зовут VERB ROOT
ангелина NOUN xcomp
компания NOUN nsubj
диджитал VERB conj
бизнес NOUN obj
звоним ADJ xcomp
вам PRON iobj
по ADP case
поводу NOUN fixed
продления NOUN nmod
лицензии NOUN nmod
а CCONJ cc
мы PRON nsubj
с ADP case
серым ADJ nmod
у ADP case
вас PRON obl
скоро ADV advmod
срок NOUN nsubj
заканчивается VERB conj


In [7]:
test_data.text[24]

'Как мне выбирать вас по каким параметрам потому что вы рассказываете я человек который 18 лет работает пролажен рассказать нужно все что угодно отработать возражения можно предложить какие то услуги выгодные'

In [63]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp = spacy.load("ru_core_news_sm")
doc = nlp(test_data.text[14])

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [57]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp = spacy.load("ru_core_news_md")
doc = nlp(sentences[0])

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
Соединённого Королевства 52 76 LOC


In [73]:
!pip3 install -c conda-forge icu=58.1

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'conda-forge'


In [82]:
!pip3 install -c conda-forge pyicu

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'conda-forge'


In [84]:
!pip3 install polyglot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
!apt-get install python3.7-numpy libicu-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python3.7-numpy
E: Couldn't find any package by glob 'python3.7-numpy'
E: Couldn't find any package by regex 'python3.7-numpy'


In [101]:
!apt-get install python3.7-icu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python3.7-icu
E: Couldn't find any package by glob 'python3.7-icu'
E: Couldn't find any package by regex 'python3.7-icu'


In [4]:
!pip3 install polyglot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
!pip3 install pycld2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
!pip3 install morfessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [115]:
!python -m pip install -U pycld2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install -U git+https://github.com/aboSamoor/polyglot.git@master

  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /private/var/folders/kb/swq539rn337_dgt2w2t4fd200000gn/T/pip-req-build-8xnu2w3f
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /private/var/folders/kb/swq539rn337_dgt2w2t4fd200000gn/T/pip-req-build-8xnu2w3f
  Resolved https://github.com/aboSamoor/polyglot.git to commit 9b93b2ecbb9ba1f638c56b92665336e93230646a
  Using cached PyICU-2.9.tar.gz (305 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 41.4 MB 1.3 MB/s eta 0:00:01    |█▉                              | 2.3 MB 1.6 MB/s eta 0:00:24     |███████████████▊                | 20.3 MB 2.3 MB/s eta 0:00:10
  Created wheel for futures: filename=futures-3.0.5-py3-none-any.whl size=14089 sha256=da0a82b097721cd040b78f4527598bae89f18b0a7f1cb0cb3f6d940d2ef7066b
  Stored in directory: /Users/ivandemyanov/Library/Caches/pip/whe

In [10]:
import polyglot
from polyglot.text import Text,Word

ImportError: cannot import name 'Locale' from 'icu' (/Users/ivandemyanov/opt/anaconda3/lib/python3.9/site-packages/icu/__init__.py)

In [6]:
import polyglot.text